In [1]:
import pandas as pd
from collections import deque, defaultdict
import heapq
import time
from concurrent.futures import ProcessPoolExecutor, as_completed

# Load the CSV file
# choice_path = '/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/Codes/Raw_Virtualized_Choices.csv'  # Update with your actual file path
choice_path = '/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/Codes/VirtualizedChoices1.csv'  # Update with your actual file path
seats_path = '/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/OfficialData/R1_App_Seats.csv'
rank_path = '/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/OfficialData/R1_App_RankDetail.csv'
candidate_profile_path = '/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/OfficialData/R1_App_CandidateProfile.csv'
quali_marks_path = '/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/OfficialData/R1_App_QualificationMarksDetail.csv'
# allotment_path = '/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/OfficialData/Neut_R1_App_Allotment_EngArc_NIC.csv' # N0te yet released
# allotment_path = "/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/OfficialData/NEUT_R1_App_Allotment_EngArc_NIC_V5.csv"
# allotment_path = "/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/OfficialData/good_data/NEUT_R1_App_Allotment_EngArc_NIC.csv"
allotment_path = "/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/OfficialData/NEUT_Official_1.csv"


/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)


In [2]:
class Program:
    def __init__(self, boardId,streamId,quotaId,instituteId,programId,categoryId,subcategoryId,genderId,
        tSeat):
        self.board_id = boardId
        self.stream = streamId
        self.quota = quotaId
        self.institute = instituteId
        self.program = programId
        self.category = categoryId
        self.subcategory = subcategoryId
        self.gender = genderId
        self.capacity = tSeat
        self.waitlist = []
        self.subCatAssign = {}

    def add_candidate(self, rk, roll_no):
        if rk and rk>0:
            heapq.heappush(self.waitlist, (-rk, roll_no))
            return True
        return False

    def remove_least_preferred(self):
        return heapq.heappop(self.waitlist)


# boardId,roundNo,streamId,groupId,seatType,quotaId,instituteId,programId,categoryId,subcategoryId,genderId,rankTypeId,tSeat,pSeat,aSeat,rSeat,vSeat
# Function to load candidates from the CSV file
def load_programs(filename):
    df = pd.read_csv(filename)
    return {(row['instituteId'], row['programId'], row['quotaId'], row['categoryId'], str(row['subcategoryId']).upper(), row['genderId']): Program(row['boardId'],
        row['streamId'],
        row['quotaId'],
        row['instituteId'],
        row['programId'],
        row['categoryId'],
        str(row['subcategoryId']).upper(),
        row['genderId'],
        row['tSeat']) for index, row in df.iterrows()}

# Load candidates from the CandidateProfile.csv file
programs = load_programs(seats_path)

print(programs)

{(2, 'ARC', '01', 1, 'NO', 'B'): <__main__.Program object at 0x75d6811259a0>, (2, 'ARC', '01', 1, 'PH', 'B'): <__main__.Program object at 0x75d686d8f410>, (2, 'ARC', '01', 2, 'NO', 'B'): <__main__.Program object at 0x75d6812c4200>, (2, 'ARC', '01', 2, 'PH', 'B'): <__main__.Program object at 0x75d6804b7ad0>, (2, 'ARC', '01', 4, 'NO', 'B'): <__main__.Program object at 0x75d680901520>, (2, 'ARC', '01', 4, 'PH', 'B'): <__main__.Program object at 0x75d680a71e20>, (2, 'ARC', '01', 5, 'NO', 'B'): <__main__.Program object at 0x75d6804ea810>, (2, 'ARC', '01', 5, 'PH', 'B'): <__main__.Program object at 0x75d6804ea210>, (2, 'ARC', '01', 6, 'NO', 'B'): <__main__.Program object at 0x75d68041e000>, (2, 'ARC', '01', 6, 'PH', 'B'): <__main__.Program object at 0x75d68041e0c0>, (2, 'ARC', '01', 7, 'NO', 'B'): <__main__.Program object at 0x75d68041e1b0>, (2, 'ARC', '01', 7, 'PH', 'B'): <__main__.Program object at 0x75d68041e240>, (2, 'ARC', '01', 35, 'NO', 'B'): <__main__.Program object at 0x75d68041ebd0

In [3]:
newProgr = {}
for program_key in programs.keys():
    program=programs[program_key]
    if program.quota=='25' and program.category==41:
        newProgr[0, program.program, program.quota, program.category, program.subcategory, program.gender]=Program(program.board_id,program.stream,program.quota,
                                                                                                0,program.program,program.category,program.subcategory,program.gender,0)
        

for program_key in newProgr.keys():
    program=newProgr[program_key]
    programs[0, program.program, program.quota, program.category, program.subcategory, program.gender]=Program(program.board_id,program.stream,program.quota,
                                                                                                0,program.program,program.category,program.subcategory,program.gender,0)

In [4]:
class Candidate:
    def __init__(self, board_id, roll_no, state_id, domicile_id, gender_id, 
                 category_id, subcat_list):
        self.board_id = board_id
        self.roll_no = roll_no
        self.state_id = state_id
        self.domicile_id = domicile_id
        self.gender_id = gender_id
        self.category_id = category_id
        self.subcat_list = subcat_list
        self.choice_list = {}
        self.choice_list_mapper = {}
        self.current_index = 1
        self.max_index = 0
        self.rank01 = None
        self.rank03 = None
        self.boardRank = None
        self.cAlloc = None

# Function to extract numbers for which "Yes" is present
def extract_yes_numbers(subCategoryList):
    items = subCategoryList.split(',')
    yes_numbers = []
    for item in items:
        key, value = item.split(':')
        if value.strip() == 'Yes' and key.isdigit():
            yes_numbers.append(int(key))
    return yes_numbers

# Function to load candidates from the CSV file
def load_candidates(filename):
    df = pd.read_csv(filename)
    return {row['rollno']: Candidate(
        row['boardId'],
        row['rollno'],
        row['stateId'],
        row['domicileId'],
        row['genderId'],
        row['categoryId'], 
        extract_yes_numbers(row["subCategoryList"])) for index, row in df.iterrows()}

candidates = load_candidates(candidate_profile_path)

In [5]:
def choiceReader(candidates, ch_df):
    for index, row in ch_df.iterrows():
        roll_no = int(row['rollno'])
        choice_no = row['ChoiceNumber']
        v_choice_no = int(row['ChoiceNumber_V'])
        inst_code = row['program_code']
        brcode = row['program_name']
        quota = row['Dom_V']
        cat = row['Cat_V']
        subcat = str(row['SubCat_V']).upper()
        gender = row['genderId']
        if gender == 1:
            gender='B'
        else:
            gender='F'
        if roll_no in candidates.keys():
            program_key = (inst_code, brcode, quota, cat, subcat, gender)
            if program_key in programs.keys():
                program = programs.get(program_key)
                candidates[roll_no].choice_list[candidates[roll_no].max_index] = program
                candidates[roll_no].choice_list_mapper[candidates[roll_no].max_index] = choice_no
                candidates[roll_no].max_index += 1
            else:
                program_key = (inst_code, brcode, quota, cat, subcat, 'B')            
                if program_key in programs.keys():
                    program = programs.get(program_key)
                    candidates[roll_no].choice_list[candidates[roll_no].max_index] = program
                    candidates[roll_no].choice_list_mapper[candidates[roll_no].max_index] = choice_no
                    candidates[roll_no].max_index += 1
    return candidates

ch_df = pd.read_csv(choice_path)
candidates = choiceReader(candidates, ch_df)            

In [6]:
df = pd.read_csv(rank_path)
# roll_no_choices = {}
for index, row in df.iterrows():
    roll_no = row['rollNo']
    if row['streamId'] == '01' or row['streamId'] == 1:
        candidates[roll_no].rank01 = row['rank']
    else:
        candidates[roll_no].rank03 = row['rank']

In [7]:
info_df = pd.read_csv(candidate_profile_path)
marks_df = pd.read_csv(quali_marks_path)

info_df = info_df.rename(columns={'rollno': 'rollNo'})
merged_df = pd.merge(marks_df, info_df, on='rollNo')
merged_df['DOB'] = pd.to_datetime(merged_df['DOB'], format='%d-%m-%Y')
merged_df['rank'] = merged_df.sort_values(by=['percentageMarks', 'DOB', 'rollNo'], ascending=[False, True, True]).groupby('roundNo_x').cumcount() + 1
ranks_dict = dict(zip(merged_df['rollNo'], merged_df['rank']))

for rollNo in ranks_dict.keys():
    candidates[rollNo].boardRank=ranks_dict[rollNo]

In [8]:
def spl_printer(roll_no):
    candidate=candidates[roll_no]
    print(f"Roll No: {candidate.roll_no}")
    print(f"  Board ID: {candidate.board_id}")
    print(f"  State ID: {candidate.state_id}")
    print(f"  Domicile ID: {candidate.domicile_id}")
    print(f"  Gender ID: {candidate.gender_id}")
    print(f"  Category ID: {candidate.category_id}")
    print(f"  Subcategory ID: {candidate.subcat_list}")
    print(f"  Choices: {candidate.max_index}")
    print(f"  Current Index: {candidate.current_index}")
    print(f"  Choice List:")
    for i in range(0, candidate.max_index):
        print(f"    {i+1}) {candidate.choice_list_mapper[i]} {candidate.choice_list[i].institute},{candidate.choice_list[i].program},{candidate.choice_list[i].category},{candidate.choice_list[i].subcategory} {candidate.choice_list[i].stream}")
    print(f"  Rank 01: {candidate.rank01}")
    print(f"  Rank 03: {candidate.rank03}")
    print(f"  Board Rank : {candidate.boardRank}")
    print()

In [9]:
# Function to print candidate details
def print_all_candidates(candidates):
    for roll_no, candidate in candidates.items():
        print(f"Roll No: {candidate.roll_no}")
        print(f"  Board ID: {candidate.board_id}")
        print(f"  State ID: {candidate.state_id}")
        print(f"  Domicile ID: {candidate.domicile_id}")
        print(f"  Gender ID: {candidate.gender_id}")
        print(f"  Category ID: {candidate.category_id}")
        print(f"  Subcat List: {candidate.subcat_list}")
        print(f"  Choice List: {candidate.max_index}")
        print(f"  Rank 01: {candidate.rank01}")
        print(f"  Rank 03: {candidate.rank03}")
        print(f"  Board Rank : {candidate.boardRank}")
        print()


# Print all candidates' details
print_all_candidates(candidates)

Roll No: 240310000014
  Board ID: 104022421
  State ID: 4.0
  Domicile ID: 4
  Gender ID: 1
  Category ID: 11
  Subcat List: []
  Choice List: 0
  Rank 01: None
  Rank 03: None
  Board Rank : 460

Roll No: 240310000054
  Board ID: 104022421
  State ID: 32.0
  Domicile ID: 32
  Gender ID: 1
  Category ID: 11
  Subcat List: []
  Choice List: 3
  Rank 01: None
  Rank 03: 153525.0
  Board Rank : 717

Roll No: 240310000639
  Board ID: 104022421
  State ID: 4.0
  Domicile ID: 4
  Gender ID: 1
  Category ID: 11
  Subcat List: []
  Choice List: 11
  Rank 01: None
  Rank 03: 169653.0
  Board Rank : 403

Roll No: 240310000878
  Board ID: 104022421
  State ID: 4.0
  Domicile ID: 4
  Gender ID: 2
  Category ID: 39
  Subcat List: [35]
  Choice List: 16
  Rank 01: None
  Rank 03: 216701.0
  Board Rank : 701

Roll No: 240310001035
  Board ID: 104022421
  State ID: 4.0
  Domicile ID: 4
  Gender ID: 1
  Category ID: 27
  Subcat List: [70]
  Choice List: 0
  Rank 01: None
  Rank 03: None
  Board Rank : 

In [10]:
def remove_candidate_with_roll_number(waitlist, roll_number):
    candidate_index = next((i for i, candidate in enumerate(waitlist) if candidate[1] == roll_number), None)
    if candidate_index is None:
        return None
    candidate = waitlist[candidate_index]
    waitlist[candidate_index] = waitlist[-1]
    waitlist.pop()
    if candidate_index < len(waitlist) and len(waitlist)>0:
        heapq._siftup(waitlist, candidate_index)
        heapq._siftdown(waitlist, 0, candidate_index)
    
    return candidate

In [11]:
def prioritizer():
    for roll in candidates.keys():
        candidate=candidates[roll]
        if (candidate.domicile_id==8 or candidate.domicile_id==9) and len(candidate.subcat_list)>0:
            if candidate.rank01:
                if candidate.subcat_list[0]>=26:
                    candidate.rank01+=10000000
                if candidate.subcat_list[0]>=27:
                    candidate.rank01+=10000000
                if candidate.subcat_list[0]>=28:
                    candidate.rank01+=10000000
            if candidate.rank03:
                if candidate.subcat_list[0]>=26:
                    candidate.rank03+=10000000
                if candidate.subcat_list[0]>=27:
                    candidate.rank03+=10000000
                if candidate.subcat_list[0]>=28:
                    candidate.rank03+=10000000
            if candidate.boardRank:
                if candidate.subcat_list[0]>=26:
                    candidate.boardRank+=10000000
                if candidate.subcat_list[0]>=27:
                    candidate.boardRank+=10000000
                if candidate.subcat_list[0]>=28:
                    candidate.boardRank+=10000000

    for roll in candidates.keys():    
        candidate3 = [c for c in candidates if candidates[c].rank03 and candidates[c].domicile_id==8]
        candidate3.sort(key=lambda c: candidates[c].rank03)
        for i, c in enumerate(candidate3, start=1):
            candidates[c].rank03 = i
        candidate1 = [c for c in candidates if candidates[c].rank01 and candidates[c].domicile_id==8]
        candidate1.sort(key=lambda c: candidates[c].rank01)
        for i, c in enumerate(candidate1, start=1):
            candidates[c].rank01 = i
        candidate4 = [c for c in candidates if candidates[c].boardRank and candidates[c].domicile_id==8]
        candidate4.sort(key=lambda c: candidates[c].boardRank)
        for i, c in enumerate(candidate4, start=1):
            candidates[c].boardRank = i
            
prioritizer()


In [12]:
def getRank(roll_no, program):
    if program.stream==3:
        return candidates[roll_no].rank03
    if program.stream==1:
        return candidates[roll_no].rank01
    elif program.stream==4: 
        return candidates[roll_no].boardRank

In [13]:
stack = []
# stack.extend(candidates)
with_rank = [c for c in candidates if candidates[c].rank03]
without_rank = [c for c in candidates if not candidates[c].rank03]
    
with_rank.sort(key=lambda c: candidates[c].rank03)

# Extend the stack with sorted candidates and then with those without rank03
stack.extend(with_rank)
stack.extend(without_rank)

In [14]:
def getLastRank(program):
    if len(program.waitlist)==0 or len(program.waitlist)<program.capacity:
        return -1
    cand=heapq.heappop(program.waitlist)
    rk=-cand[0]
    heapq.heappush(program.waitlist, cand)
    return rk

In [15]:
def checker(candidate):
    for i in range (1, candidate.current_index):
        preference=candidate.choice_list[i-1]
        if preference :
            preference = programs[preference.institute, preference.program, preference.quota, preference.category, preference.subcategory, preference.gender]
            rk = getRank(candidate.roll_no, preference)
            if rk and rk>0 and preference.capacity>0:
                if preference.quota!='25' or preference.category!=41:
                    if preference.capacity>len(preference.waitlist) or getLastRank(preference)>rk:
                        return True
                else:
                    if len(candidate.subcat_list)<1:
                        continue
                    else:
                        subcatProg=programs[0, preference.program, preference.quota, preference.category, preference.subcategory, preference.gender]
                        if candidate.subcat_list[0] not in subcatProg.subCatAssign.keys():
                            if len(preference.waitlist)<preference.capacity:
                                return True
                            elif getLastRank(subcatProg)>rk and getLastRank(preference)>rk:
                                return True
                        else:
                            catCandidate=candidates[subcatProg.subCatAssign[candidate.subcat_list[0]]]
                            if getRank(catCandidate.roll_no, preference)>rk:
                                if len(preference.waitlist)<preference.capacity:                                    
                                    return True
                                elif getLastRank(preference)>rk:
                                    return True
                            else:
                                continue
    return False
            

In [16]:
def alloter():
    while stack:
        candidate = stack.pop()
        candidate = candidates[candidate]
        if candidate.current_index<=candidate.max_index:
            preference = candidates[candidate.roll_no].choice_list[candidate.current_index-1]
            if preference :
                preference = programs[preference.institute, preference.program, preference.quota, preference.category, preference.subcategory, preference.gender]
                rk = getRank(candidate.roll_no, preference)
                if rk and rk>0 and preference.capacity>0:
                    done=False
                    if candidate.domicile_id==41 and candidate.quota==25:
                        done=checker(candidate)
                    if done and candidate.current_index!=1:
                        candidate.current_index=1
                        stack.append(candidate.roll_no)
                        continue
                    if preference.quota!='25' or preference.category!=41:
                        if preference.capacity>len(preference.waitlist) or getLastRank(preference)>rk:
                            preference.add_candidate(rk, candidate.roll_no)
                            candidates[candidate.roll_no].cAlloc = preference
                            if len(preference.waitlist) > preference.capacity:
                                _, roll_no = heapq.heappop(preference.waitlist)
                                if candidate.category_id==41 and candidate.domicile_id==25:
                                    candidates[roll_no].current_index = 1  
                                else:
                                    candidates[roll_no].current_index += 1 
                                candidates[roll_no].cAlloc=None
                                stack.append(roll_no)
                        else:
                            candidates[candidate.roll_no].current_index += 1
                            stack.append(candidate.roll_no)        
                    else:
                        if len(candidate.subcat_list)<1:
                            candidates[candidate.roll_no].current_index += 1
                            stack.append(candidate.roll_no)    
                        else:
                            subcatProg=programs[0, preference.program, preference.quota, preference.category, preference.subcategory, preference.gender]
                            if candidate.subcat_list[0] not in subcatProg.subCatAssign.keys():
                                if len(preference.waitlist)<preference.capacity:
                                    preference.add_candidate(rk, candidate.roll_no)
                                    subcatProg.add_candidate(rk, candidate.roll_no)
                                    candidate.cAlloc=preference
                                    subcatProg.subCatAssign[candidate.subcat_list[0]]=candidate.roll_no
                                elif getLastRank(subcatProg)>rk and getLastRank(preference)>rk:
                                    _, catCandidate=subcatProg.remove_least_preferred()
                                    catCandidate=candidates[catCandidate]
                                    remove_candidate_with_roll_number(catCandidate.cAlloc.waitlist, catCandidate.roll_no)
                                    catCandidate.cAlloc=None
                                    catCandidate.current_index = 1
                                    stack.append(catCandidate.roll_no)    
                                    ns = {}
                                    for sc in subcatProg.subCatAssign.keys():
                                        if sc!=catCandidate.subcat_list[0]:
                                            ns[sc]=subcatProg.subCatAssign[sc]
                                    subcatProg.subCatAssign=ns
                                    if len(preference.waitlist)==preference.capacity and len(preference.waitlist)>0:
                                        _, catCandidate1=preference.remove_least_preferred()
                                        remove_candidate_with_roll_number(subcatProg.waitlist, catCandidate1)
                                        candidates[catCandidate1].cAlloc=None
                                        candidates[catCandidate1].current_index = 1
                                        stack.append(catCandidate1)
                                        ns = {}
                                        for sc in subcatProg.subCatAssign.keys():
                                            if sc!=candidates[catCandidate1].subcat_list[0]:
                                                ns[sc]=subcatProg.subCatAssign[sc]
                                        subcatProg.subCatAssign=ns
                                    preference.add_candidate(rk, candidate.roll_no)
                                    subcatProg.add_candidate(rk, candidate.roll_no)
                                    candidate.cAlloc=preference
                                    subcatProg.subCatAssign[candidate.subcat_list[0]]=candidate.roll_no
                            else:
                                catCandidate=candidates[subcatProg.subCatAssign[candidate.subcat_list[0]]]
                                if getRank(catCandidate.roll_no, preference)>rk:
                                    remove_candidate_with_roll_number(catCandidate.cAlloc.waitlist, catCandidate.roll_no)
                                    remove_candidate_with_roll_number(subcatProg.waitlist, catCandidate.roll_no)
                                    catCandidate.cAlloc=None
                                    catCandidate.current_index = 1
                                    stack.append(catCandidate.roll_no)
                                    if len(preference.waitlist)<preference.capacity:
                                        # remove_candidate_with_roll_number(catCandidate.cAlloc.waitlist, catCandidate.roll_no)
                                        # remove_candidate_with_roll_number(subcatProg.waitlist, catCandidate.roll_no)
                                        # catCandidate.cAlloc=None
                                        # catCandidate.current_index = 1
                                        # stack.append(catCandidate.roll_no)    
                                        preference.add_candidate(rk, candidate.roll_no)
                                        subcatProg.add_candidate(rk, candidate.roll_no)
                                        candidate.cAlloc=preference
                                        subcatProg.subCatAssign[candidate.subcat_list[0]]=candidate.roll_no
                                    elif getLastRank(preference)>rk and len(preference.waitlist)>0:
                                        _, catCandidate1=preference.remove_least_preferred()
                                        remove_candidate_with_roll_number(subcatProg.waitlist, catCandidate1)
                                        candidates[catCandidate1].cAlloc=None
                                        candidates[catCandidate1].current_index = 1
                                        stack.append(catCandidate1)
                                        ns = {}
                                        for sc in subcatProg.subCatAssign.keys():
                                            if sc!=candidates[catCandidate1].subcat_list[0]:
                                                ns[sc]=subcatProg.subCatAssign[sc]
                                        subcatProg.subCatAssign=ns
                                        catCandidate=subcatProg.subCatAssign[candidate.subcat_list[0]]
                                        preference.add_candidate(rk, candidate.roll_no)
                                        subcatProg.add_candidate(rk, candidate.roll_no)
                                        candidate.cAlloc=preference
                                        subcatProg.subCatAssign[candidate.subcat_list[0]]=candidate.roll_no   
                                    else:                             
                                        catCandidate = stack.pop()
                                        catCandidate = candidates[catCandidate]
                                        preference.add_candidate(getRank(catCandidate.roll_no, preference), catCandidate.roll_no)
                                        subcatProg.add_candidate(getRank(catCandidate.roll_no, preference), catCandidate.roll_no)
                                        catCandidate.cAlloc=preference
                                        for i in range(0, catCandidate.max_index):
                                            if catCandidate.choice_list[i]==preference:
                                                catCandidate.current_index=i+1
                                                break

                            if not candidate.cAlloc:
                                candidate.current_index += 1
                                stack.append(candidate.roll_no)
                            
                else:
                    candidates[candidate.roll_no].current_index += 1
                    candidates[candidate.roll_no].cAlloc=None
                    stack.append(candidate.roll_no)


alloter()

In [17]:
for i in range (0, len(candidates)):
    for roll in candidates.keys():
        candidate=candidates[roll]
        if candidate.domicile_id==25 and candidate.category_id==41 and checker(candidate):
            if candidate.cAlloc:
                remove_candidate_with_roll_number(candidate.cAlloc.waitlist, candidate.roll_no)
                if candidate.cAlloc.category==41:
                    subcatProg=programs[0, candidate.cAlloc.program, candidate.cAlloc.quota, candidate.cAlloc.category, candidate.cAlloc.subcategory, candidate.cAlloc.gender]
                    remove_candidate_with_roll_number(subcatProg.waitlist, candidate.roll_no)
                    ns = {}
                    for sc in subcatProg.subCatAssign.keys():
                        if sc!=candidate.subcat_list[0]:
                            ns[sc]=subcatProg.subCatAssign[sc]
                    subcatProg.subCatAssign=ns
            candidates[roll].cAlloc=None
            print(candidate.roll_no)
            candidates[roll].current_index=1
            stack.append(candidate.roll_no)
            alloter()

240310225846
240311169615
240320197358


In [18]:
# if len(preference.waitlist) < preference.capacity:
#                                 for subCat in candidate.subcat_list:
#                                     if subCat not in preference.subCatAssign.keys():
#                                         programs[preference.institute, preference.program, preference.quota, preference.category, preference.subcategory, preference.gender].subCatAssign[subCat]=candidate.roll_no
#                                         preference.add_candidate(rk, candidate.roll_no)
#                                         candidate.cAlloc=preference
#                                         break
#                             if candidate.cAlloc!=None:
#                                 continue
#                             found=True
#                             for sc in preference.subCatAssign.keys():
#                                 if sc in candidate.subcat_list:
#                                     if getRank(preference.subCatAssign[sc], preference)>rk:
#                                         removal_cand=preference.subCatAssign[sc]
#                                         remove_candidate_with_roll_number(preference.waitlist, removal_cand)
#                                         candidates[removal_cand].cAlloc=None
#                                         candidates[removal_cand].current_index=1
#                                         programs[preference.institute, preference.program, preference.quota, preference.category, preference.subcategory, preference.gender].subCatAssign[sc]=candidate.roll_no
#                                         candidate.cAlloc=preference
#                                         preference.add_candidate(rk, candidate.roll_no)
#                                         stack.append(removal_cand)
#                                         break
#                                     else:
#                                         candidates[candidate.roll_no].current_index += 1
#                                         stack.append(candidate.roll_no)
#                                         found=False
#                                         break
#                             if candidate.cAlloc==None and found:
#                                 if rk<getLastRank(preference):
#                                 # if candidate.subcat_list[0]<max(preference.subCatAssign.keys()):
#                                     preference.add_candidate(rk, candidate.roll_no)
#                                     programs[preference.institute, preference.program, preference.quota, preference.category, preference.subcategory, preference.gender].subCatAssign[candidate.subcat_list[0]]=candidate.roll_no
#                                     candidates[candidate.roll_no].cAlloc = preference
#                                     roll_no=preference.subCatAssign[max(preference.subCatAssign.keys())]
#                                     _, roll_no = heapq.heappop(preference.waitlist)                        
#                                     # remove_candidate_with_roll_number(preference.waitlist, roll_no)
#                                     candidates[roll_no].current_index = 1 
#                                     candidates[roll_no].cAlloc=None
#                                     # if candidates[roll_no].subcat_list[0] in programs[preference.institute, preference.program, preference.quota, preference.category, preference.subcategory, preference.gender].subCatAssign.keys():
#                                     del programs[preference.institute, preference.program, preference.quota, preference.category, preference.subcategory, preference.gender].subCatAssign[candidates[roll_no].subcat_list[0]]
#                                     stack.append(roll_no)
#                                 else:
#                                     candidates[candidate.roll_no].current_index += 1
#                                     candidates[candidate.roll_no].cAlloc=None
#                                     stack.append(candidate.roll_no)

In [19]:
print("Final Allocations:")
for roll_no, candidate in candidates.items():
    if candidate.cAlloc:
        print(f"Candidate {roll_no} is allocated to program  Choice - {candidate.current_index}")

sum=0
print("\nFinal Waitlists:")
for program_key, program in programs.items():
    sum=sum+len(program.waitlist)
    if len(program.waitlist)>0:
        print(f"Program {program_key} with {program.capacity} waitlist: {program.waitlist} with {program.capacity - len(program.waitlist)} remaining seats")

    

Final Allocations:
Candidate 240310001476 is allocated to program  Choice - 5
Candidate 240310002746 is allocated to program  Choice - 1
Candidate 240310002807 is allocated to program  Choice - 1
Candidate 240310003114 is allocated to program  Choice - 40
Candidate 240310003423 is allocated to program  Choice - 2
Candidate 240310004318 is allocated to program  Choice - 15
Candidate 240310005241 is allocated to program  Choice - 25
Candidate 240310005570 is allocated to program  Choice - 1
Candidate 240310007754 is allocated to program  Choice - 3
Candidate 240310008246 is allocated to program  Choice - 17
Candidate 240310008425 is allocated to program  Choice - 3
Candidate 240310009097 is allocated to program  Choice - 7
Candidate 240310016941 is allocated to program  Choice - 29
Candidate 240310018415 is allocated to program  Choice - 6
Candidate 240310019155 is allocated to program  Choice - 3
Candidate 240310021411 is allocated to program  Choice - 14
Candidate 240310023939 is alloc

In [20]:
rem_sum=0
for program_key, program in programs.items():
    rem_sum=rem_sum+program.capacity-len(program.waitlist)
    # if program.capacity-len(program.waitlist)>0:
    print(f"Program {program_key} with {program.capacity} has {program.capacity - len(program.waitlist)} remaining seats. Stream : {program.stream}")
        

Program (2, 'ARC', '01', 1, 'NO', 'B') with 0 has 0 remaining seats. Stream : 1
Program (2, 'ARC', '01', 1, 'PH', 'B') with 0 has 0 remaining seats. Stream : 1
Program (2, 'ARC', '01', 2, 'NO', 'B') with 0 has 0 remaining seats. Stream : 1
Program (2, 'ARC', '01', 2, 'PH', 'B') with 0 has 0 remaining seats. Stream : 1
Program (2, 'ARC', '01', 4, 'NO', 'B') with 0 has 0 remaining seats. Stream : 1
Program (2, 'ARC', '01', 4, 'PH', 'B') with 0 has 0 remaining seats. Stream : 1
Program (2, 'ARC', '01', 5, 'NO', 'B') with 0 has 0 remaining seats. Stream : 1
Program (2, 'ARC', '01', 5, 'PH', 'B') with 0 has 0 remaining seats. Stream : 1
Program (2, 'ARC', '01', 6, 'NO', 'B') with 0 has 0 remaining seats. Stream : 1
Program (2, 'ARC', '01', 6, 'PH', 'B') with 0 has 0 remaining seats. Stream : 1
Program (2, 'ARC', '01', 7, 'NO', 'B') with 2 has 2 remaining seats. Stream : 1
Program (2, 'ARC', '01', 7, 'PH', 'B') with 0 has 0 remaining seats. Stream : 1
Program (2, 'ARC', '01', 35, 'NO', 'B') 

In [21]:
import csv

unallocated = []

with open('Smolfinal_allocations.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Roll No", "Program Code", "Branch Code", "Quota", "Category", "Subcategory", "Gender", "Rank", "ChoiceIndex", "VirtualIndex"])
    for roll_no, candidate in candidates.items():
        if not candidate.cAlloc:
            unallocated.append(candidate.roll_no)
        else:
            ch=candidate.current_index-1
            if ch not in candidate.choice_list_mapper.keys():
                ch=candidate.max_index
            else:
                ch=candidate.choice_list_mapper[ch]
            writer.writerow([
                roll_no,
                candidate.cAlloc.institute,
                candidate.cAlloc.program,
                candidate.cAlloc.quota,
                candidate.cAlloc.category,
                candidate.cAlloc.subcategory,
                candidate.cAlloc.gender,
                getRank(roll_no, candidate.cAlloc),
                ch,
                candidate.current_index-1
            ])

print("Allocations generated")

# with open('unallocated_candidates.csv', mode='w', newline='') as file:
with open('final_unallocated_candidates.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Roll No"])
    for roll_no in unallocated:
        writer.writerow([roll_no])

print("Unallocations generated")

Allocations generated
Unallocations generated


In [22]:
# def allotReader(all_df):
#     for index, row in all_df.iterrows():
#         candidate=candidates[row['rollno']]
#         # ch=candidate.current_index-1
#         # if ch not in candidate.choice_list_mapper.keys():
#         #         if candidate.cAlloc:
#         #                 ch=candidate.max_index
#         #         else:
#         #                ch=-1
#         # else:
#         #         ch=candidate.choice_list_mapper[ch]
#         done.append(row['rollno'])
#         spli = row['allottedSequence'].split('-')
#         if len(spli)!=5:
#                continue
#         if candidate.current_index-1!=row['virtualOptNo']:
#                 if candidate.cAlloc:
#                         # print(f"{len(spli)} {int(spli[0])} {spli[1]} {spli[2]} {int(spli[3])} {spli[4].upper()}")
#                         if len(spli)!=5 or candidate.cAlloc.institute != int(spli[0]) or candidate.cAlloc.program != spli[1] or candidate.cAlloc.quota != spli[2] or candidate.cAlloc.category != int(spli[3]) or candidate.cAlloc.subcategory.upper() != spli[4].upper():
#                                 print(f'{candidate.roll_no} got {candidate.subcat_list}({candidate.choice_list_mapper[candidate.current_index-1]})({candidate.cAlloc.institute},{candidate.cAlloc.program},{candidate.cAlloc.quota},{candidate.cAlloc.category},{candidate.cAlloc.subcategory.upper()}) but ({row['optionNo']})({int(spli[0])},{spli[1]},{spli[2]},{int(spli[3])},{spli[4].upper()}) in NIC {getRank(candidate.roll_no, candidate.cAlloc)} and {row['rank']}') 

#                 else:
#                         print(f"{candidate.roll_no} didn't get in ours but got ({row['optionNo']})({int(spli[0])},{spli[1]},{spli[2]},{int(spli[3])},{spli[4].upper()}) in NIC's {row['rank']}")

# all_df = pd.read_csv(allotment_path)
# done = []
# allotReader(all_df)
# for roll in candidates.keys():
#        if not roll in done and candidates[roll].cAlloc:
#               candidate=candidates[roll]
#               print(f'{candidate.roll_no} got {candidate.subcat_list}({candidate.choice_list_mapper[candidate.current_index-1]})({candidate.cAlloc.institute},{candidate.cAlloc.program},{candidate.cAlloc.quota},{candidate.cAlloc.category},{candidate.cAlloc.subcategory.upper()}) but not in NIC {getRank(candidate.roll_no, candidate.cAlloc)}') 

In [23]:
def allotReader(all_df):
    for index, row in all_df.iterrows():
        candidate=candidates[row['rollNo']]
        done.append(row['rollNo'])
        # spli = row['allottedSequence'].split('-')
        # if len(spli)!=5:
        #        continue
        # if candidate.current_index-1!=row['virtualOptNo']:
        if row['allottedQuota'][:2]=='01':
               continue
        if candidate.cAlloc:
                if candidate.choice_list_mapper[candidate.current_index-1]!=row['optionNo']:
                # boardId,roundNo,rollNo,otherInfo,ranktypeId,rank,optionNo,instituteId,programId,streamId,groupId,seatTypeId,allottedCat,allottedQuota,seatGenderId
                # print(f"{len(spli)} {int(spli[0])} {spli[1]} {spli[2]} {int(spli[3])} {spli[4].upper()}")
                # if len(spli)!=5 or candidate.cAlloc.institute != int(spli[0]) or candidate.cAlloc.program != spli[1] or candidate.cAlloc.quota != spli[2] or candidate.cAlloc.category != int(spli[3]) or candidate.cAlloc.subcategory.upper() != spli[4].upper():
                        if candidate.cAlloc.institute != row['instituteId'] or candidate.cAlloc.program != row['programId'] or candidate.cAlloc.quota != row['allottedQuota'] or candidate.cAlloc.category != row['allottedCat'][:2] or candidate.cAlloc.subcategory.upper() != row['allottedCat'][2:].upper():
                                print(f'{candidate.roll_no} got {candidate.subcat_list}({candidate.choice_list_mapper[candidate.current_index-1]})({candidate.cAlloc.institute},{candidate.cAlloc.program},{candidate.cAlloc.quota},{candidate.cAlloc.category},{candidate.cAlloc.subcategory.upper()}) but ({row['optionNo']})({row['instituteId']},{row['programId']},{row['allottedQuota']},{row['allottedCat'][:2]},{row['allottedCat'][2:]}) in NIC {getRank(candidate.roll_no, candidate.cAlloc)} and {row['rank']}') 

        else:
                print(f"{candidate.roll_no} didn't get in ours but got ({row['optionNo']})({row['instituteId']},{row['programId']},{row['allottedQuota']},{row['allottedCat'][:2]},{row['allottedCat'][2:]}) in NIC's {row['rank']}")

all_df = pd.read_csv(allotment_path)
done = []
allotReader(all_df)
for roll in candidates.keys():
       if not roll in done and candidates[roll].cAlloc:
              # if candidates[roll].cAlloc.program=='PHA' or candidates[roll].cAlloc.program=='BPH':
              #        continue
              candidate=candidates[roll]
              print(f'{candidate.roll_no} got {candidate.subcat_list}({candidate.choice_list_mapper[candidate.current_index-1]})({candidate.cAlloc.institute},{candidate.cAlloc.program},{candidate.cAlloc.quota},{candidate.cAlloc.category},{candidate.cAlloc.subcategory.upper()}) but not in NIC {getRank(candidate.roll_no, candidate.cAlloc)}') 

In [24]:
for key in newProgr.keys():
    # for cat in programs[key].subCatAssign.keys():
    for cat in sorted(programs[key].subCatAssign.keys()):
        print(f"{candidates[programs[key].subCatAssign[cat]].cAlloc.institute} {key[1]} {cat} - {programs[key].subCatAssign[cat]} - {getRank(programs[key].subCatAssign[cat],programs[key])}")

69 CVE 1 - 240311109892 - 800721.0
106 CVE 3 - 240320020528 - 465373.0
59 CVE 6 - 240311311718 - 508355.0
59 CVE 7 - 240311089558 - 524618.0
243 CVE 10 - 240320197358 - 800378.0
17 EEE 5 - 240310270283 - 1015417.0
21 CSS 5 - 240310225846 - 964405.0
237 ELE 2 - 240310419341 - 616156.0
167 CTS 2 - 240310378347 - 651352.0
